In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

Мероприятия

In [4]:
events = pd.read_csv('drive/My Drive/Colab Notebooks/hackaton/events.csv')

In [5]:
events=events.fillna('')

In [6]:
events.head(2)

,id,Название мероприятия,Статус,Наименование учреждения,Тип учреждения,Подразделение,Стоимость (полная),Тип мероприятия,Направленность мероприятия,Мероприятие относится к ежегодным праздникам и общегородским мероприятиям,Наименование праздника или события,Дата начала мероприятия,Время начала мероприятия,Дата окончания мероприятия,Время окончания мероприятия,Краткое описание,Проводится онлайн,Место проведения,Округ,Район,Доступность мероприятия для лиц с ОВЗ,Тип ОВЗ,Возрастной ценз участников мероприятия,Возрастная категория,Целевая аудитория
0,5,Фестивлаь современных молодежных уличных культ...,Завершено,ГБУК г. Москвы «Клуб «Современник»,Дома культуры,,бесплатное,фестиваль,культурно-досуговое,,,13.06.2020 16:00,,13.06.2020 20:00,,Клуб Современник проведет фестиваль современны...,Нет,,,,недоступно,,0+,от 20 до 40,
1,12,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",Завершено,ГБУК г.Москвы «ДК «Нагорный»,Дома культуры,,бесплатное,"танцевальный вечер, бал",патриотическое,праздник,9 мая. День Победы,09.05.2020 14:30,,09.05.2020 17:30,,"Танцевальный вечер ""Ретро"" в честь Дня Победы",Нет,"Электролитный пр. 3, корпус 1",Южный административный округ,Нагорный,недоступно,,16+,от 40 до 85,"работающая молодежь, взрослые, пенсионеры"


In [7]:
events['text']=events['Название мероприятия']+' '+events['Наименование учреждения']+' '+events['Направленность мероприятия']+' '+events['Краткое описание']

**Кружки**

In [8]:
hobbies = pd.read_csv('drive/My Drive/Colab Notebooks/hackaton/club_type.csv',sep=';')

In [9]:
hobbies.drop(columns='Unnamed: 0', inplace=True)
hobbies.columns = ['id_classificator', 'id_parent', 'hobbie']

In [82]:
hobbies.head(10)

,id_classificator,id_parent,hobbie,text
0,3002985,,Автошкола,Автошкола
1,3002986,3.00298e+06,Автоклуб,Автоклуб
2,3002987,3.00298e+06,Правила дорожного движения,Правила дорожного движения
3,3002988,3.00298e+06,Юный водитель,Юный водитель
4,3002989,,Архитектура,Архитектура
5,3002990,3.00299e+06,Основы архитектуры,Основы архитектуры
6,3002991,,Военная история,Военная история
7,3002992,3.00299e+06,Военно-историческая реконструкция,Военно-историческая реконструкция
8,3002993,3.00299e+06,Военно-историческое моделирование,Военно-историческое моделирование
9,3002994,3.00299e+06,Военно-патриотический клуб,Военно-патриотический клуб


In [11]:
hobbies=hobbies.fillna('')

In [12]:
hobbies['text']=hobbies['hobbie']

**Книги**

In [13]:
books = pd.read_csv('drive/My Drive/Colab Notebooks/hackaton/books_catalog_.csv', sep='\t')

In [14]:
books.head(10)

,doc_id,p100a,p245a,p260a,p260b,p260c,p490a,p650a,p084a,p521a
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+
1,2,NaN,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,NaN,Наука. Науковедение,72,12+
2,3,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
3,4,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
4,5,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
5,6,Тараканов Борис Игоревич,Кольцо времени,Москва,АиФ Принт,2001,Черная звезда,Художественная литература ; Фэнтэзи,84(2Рос)6,NaN
6,8,NaN,Фантастика-86,Москва,Молодая гвардия,1986,NaN,Художественная литература,84(2Рос)6,NaN
7,9,NaN,Фантастика-90,Москва,Молодая гвардия,1990,NaN,Художественная литература ; Фантастика,84(2Рос)6,NaN
8,10,Федотова Надежда Григорьевна,Вождей не выбирают,Москва,Армада ; Альфа-книга,2005,NaN,Художественная литература ; Художественно-доку...,84(2Рос)6,NaN
9,11,Федорова Екатерина,Леди-рыцарь,Москва,Армада ; Альфа-книга,2002,NaN,Художественная литература ; Фантастика,84(2Рос)6,NaN


In [15]:
books = books[['doc_id', 'p100a', 'p245a', 'p650a']]

In [16]:
books.columns = ['book_id', 'author', 'name', 'genre']

In [17]:
books=books.fillna('')

In [18]:
books.head(5)

,book_id,author,name,genre
0,1,Устинова Татьяна Витальевна,Призрак Канта,Художественная литература ; Российский детектив
1,2,,Наука и жизнь,Наука. Науковедение
2,3,,Наука и жизнь,Наука. Науковедение
3,4,,Наука и жизнь,Наука. Науковедение
4,5,,Наука и жизнь,Наука. Науковедение


In [19]:
books['text']=books['author']+' '+books['name']

**Сборка в единый датасет**

In [20]:
pdList = [events[['id', 'text']], hobbies[['id_classificator', 'text']], books[['book_id', 'text']]]  # List of your dataframes
df = pd.concat(pdList)

In [21]:
df

,id,text,id_classificator,book_id
0,5.0,Фестивлаь современных молодежных уличных культ...,NaN,NaN
1,12.0,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",NaN,NaN
2,3179.0,Врасплох ГБУК г. Москвы «ДК «Нагатино» антитер...,NaN,NaN
3,486.0,Праздник Новогодней елки. Новогодний концерт в...,NaN,NaN
4,2100.0,"Выставка к Рождеству ""Сувенир Года"" ГБУК г.Мос...",NaN,NaN
...,...,...,...,...
1370452,NaN,Золотой век русской поэзии [Текст],NaN,1783623.0
1370453,NaN,Антология русского лиризма. XX век,NaN,1783624.0
1370454,NaN,Кэрролл Льюис Приключения Алисы,NaN,1783625.0
1370455,NaN,Барб-Галль Франсуаза Шедевры мировой живописи....,NaN,1783626.0


**Предобработка текста**

In [22]:
!pip install nltk

In [23]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re


#Create lemmatizer and stopwords list
mystem = Mystem() 
stemmer = SnowballStemmer('russian')
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
#Preprocess function
def preprocess_text(text):
    
    tokens = re.sub(r'[^A-Za-zА-Яа-яёЁ0-9-\s]',r'',str(text).lower())
    #tokens = mystem.lemmatize(text.lower())
    tokens = tokens.split(' ')
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    #text = " ".join(tokens)
    
    tokens = [stemmer.stem(word) for word in tokens] #??????????

    return tokens

In [25]:
def preprocess_text_easy(text):
    
    tokens = re.sub(r'[^a-zа-яё0-9-\s]',r'',str(text).lower())

    return tokens

In [26]:
preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")


['сказа', 'виж', 'кто-т', 'наступ', 'грабл', 'разочарова', 'натравл']

In [27]:
text_new =df["text"].apply(preprocess_text)

In [28]:
#text_easy = df["text"].apply(preprocess_text_easy)

In [29]:
df['text_new'] = text_new
#df['text_easy'] = text_easy

In [30]:
df.head()

,id,text,id_classificator,book_id,text_new
0,5.0,Фестивлаь современных молодежных уличных культ...,NaN,NaN,"[фестивла, современ, молодежн, уличн, культур,..."
1,12.0,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",NaN,NaN,"[танцевальн, вечер, ретр, чест, дня, побед, ва..."
2,3179.0,Врасплох ГБУК г. Москвы «ДК «Нагатино» антитер...,NaN,NaN,"[врасплох, гбук, г, москв, дк, нагатин, антите..."
3,486.0,Праздник Новогодней елки. Новогодний концерт в...,NaN,NaN,"[праздник, новогодн, елк, новогодн, концерт, г..."
4,2100.0,"Выставка к Рождеству ""Сувенир Года"" ГБУК г.Мос...",NaN,NaN,"[выставк, рождеств, сувенир, год, гбук, гмоскв..."


In [31]:
from gensim.models import Word2Vec
from gensim.models import FastText


In [32]:

model = FastText(sentences=df['text_new'], size=256, window=3)


In [33]:
from gensim.test.utils import get_tmpfile

In [34]:
#fname = get_tmpfile('drive/My Drive/Colab Notebooks/hackaton/fasttext.model')
#model.save(fname)

In [35]:
model.save('drive/My Drive/Colab Notebooks/hackaton/fasttext.model')

In [73]:
embeddings=[]

In [80]:
model.wv['Фестивлаь современных молодежных уличных культ...	']

array([ 0.36141336, -0.25877863, -0.69733125, -0.55912435, -0.4816257 ,
       -0.26052102, -0.31252518,  0.44131243,  0.1832992 , -0.7832446 ,
        0.5063123 ,  0.03259745, -0.1222129 , -0.44227058, -0.39789394,
        0.6299624 ,  0.19918299, -0.75434434, -0.13582578, -0.25081116,
       -0.21066542, -0.2875092 ,  0.06249294, -0.13586351, -0.4715543 ,
        0.40079284,  0.20185131,  0.19646426, -0.48713905,  0.10241161,
        0.14512266,  0.12900306, -0.2046513 ,  0.14580221,  0.3844039 ,
        0.05120646,  0.03133939,  0.06500049,  0.10541821,  0.10923085,
       -0.713263  ,  1.1951798 , -0.61891776,  0.6365845 , -0.22062738,
        0.01535683,  0.6391596 ,  0.00308906,  1.2049077 , -0.01622031,
        0.24946725,  0.09756835, -0.18851706, -0.45716026, -0.53315437,
       -0.41966698, -0.84513587,  0.25634578, -0.26321197, -0.15034294,
        0.63398916,  0.28517762, -0.2807712 , -0.2838641 ,  0.20320325,
        0.23273583,  0.27117845, -0.32710353, -0.7172847 ,  0.22

In [37]:
model2 = FastText.load('drive/My Drive/Colab Notebooks/hackaton/fasttext.model')